In [21]:
cd ..

/home/dmitriishubin/Desktop


In [ ]:
#### insert the name of the last run
!tensorboard --logdir=runs/May11_01-21-42_dmitrii

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import gc
import ast
import json
import os

from metrics import Metric
from utils.KPI_plots import plot_confusion_matrix

In [3]:
%matplotlib qt

# Settings

In [18]:
competition_metric = Metric()

# Dataset A

In [19]:
DEBUG_PATH = './data/CV_debug/A/'
DATA_PATH = './data/A/formatted/'

list_records = [i[:-4] for i in os.listdir(DEBUG_PATH) if i.find('.npy')!=-1]


In [ ]:
def load_waveform(data_path,name):
    
    signal = np.load(data_path+name+'.npy')
        
    return signal

def load_label(data_path,name):
    
    label = json.load(open(data_path+name+'.npy'))
        
    return label

In [ ]:
# Main processing pipeline
scores_errors = np.array([])
scores_competition = np.array([])
records = np.array([]).astype(np.str)
labels = np.array([])
preds = np.array([])

#load all records
for record in list_records:
    
    label = load_label(DATA_PATH,record)
    records = np.append(records,label['filename'])
    label = label['labels_training_merged']
    
    preds = load_label(DEBUG_PATH,record)
    preds = preds['predicted_label']
    
    #calc score for each record
    scores_competition = np.append(scores_competition,competition_metric.compute(labels, preds))
    scores_errors = np.append(scores_errors,np.sum(np.abs(np.array(label) - np.array(preds))))
    
    #add predictions and labels for overall KPI estimation
    labels = np.append(labels,np.array(label))
    preds = np.append(preds,np.array(pred))

#plot modified consustion matrix
matrix = competition_metric.compute_modified_confusion_matrix(labels, preds)
Labal_names = ['IAVB'
'AF',
'AFL',
'Brady',
'CRBBB',
'IRBBB',
'LAnFB',
'LAD',
'LBBB',
'LQRSV',
'NSIVCB',
'PR',
'PAC',
'PVC',
'LPR',
'LQT',
'QAb',
'RAD',
'RBBB',
'SA',
'SB',
'SNR',
'STach',
'SVPB',
'TAb',
'TInv',
'VP'
 ]


plot_confusion_matrix(matrix,Labal_names)

In [ ]:
#keeep records with errors only
records = records[np.where(scores_errors > 0)]
scores_errors = scores_errors[np.where(scores_errors > 0)]

#TODO: clarify the type of sort
score_order = np.argsort(scores_errors)
records = records[score_order[::1]]

#save a list of files in csv
pd.DataFrame(records).to_csv('./data/')

In [ ]:
def plot_record(record,data):
    
    ecg = np.load(DATA_PATH+record+'.npy')
    ecg = preprocessing.run(ecg)
    meta = json.load(open(DATA_PATH+ f'{record}.json'))
    
    heatmap = np.array(data[record]['heatmap'],dtype=np.float)
    
    #plot the data
    fig,ax = plt.subplots(figsize=(20,20))
    fig.suptitle(record+', labels: '+str(meta['labels']))
    for i in range(12):
        ax.plot(ecg[:,i]-2.5*i)
    ax.plot(heatmap-2.5*12)
    plt.show()
    
    return 0

plot_record(record=records[0],data=data)